Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [4]:
path = "../"
df = pd.read_pickle(f"{path}output/pickle_results/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])[["kmeans_ari_0", "leiden_ari_0"]].mean().unstack("layers")

kmeans_ari_0                                    \
layers                          [100, 30] [200, 100, 30, 30] [200, 100, 30]   
temperature dropout lr                                                        
0.01        0.2     0.000001     0.451912           0.309376       0.384112   
                    0.000010     0.451912           0.309376       0.384112   
                    0.000100     0.451912           0.309376       0.384112   
            0.5     0.000001     0.520133           0.478403       0.416010   
                    0.000010     0.520133           0.478403       0.416010   
                    0.000100     0.520133           0.478403       0.416010   
            0.8     0.000001     0.619568           0.598459       0.607688   
                    0.000010     0.619568           0.598459       0.607688   
                    0.000100     0.619568           0.598459       0.607688   
            0.9     0.000001     0.754953           0.595293       0.685175   
                    0.000010     0.754953           0.595293       0.685175   
                    0.000100     0.754953           0.595293       0.685175   
0.07        0.2     0.000001     0.133621           0.083244       0.096137   
                    0.000010     0.133621           0.083244       0.096137   
                    0.000100     0.133621           0.083244       0.096137   
            0.5     0.000001     0.344110           0.309458       0.343249   
                    0.000010     0.344110           0.309458       0.343249   
                    0.000100     0.344110           0.309458       0.343249   
            0.8     0.000001     0.582870           0.602947       0.594464   
                    0.000010     0.605162           0.621365       0.610322   
                    0.000100     0.605162           0.621365       0.610322   
            0.9     0.000001     0.662918           0.657501       0.605957   
                    0.000010     0.662918           0.657501       0.605957   
                    0.000100     0.662918           0.657501       0.605957   
0.50        0.2     0.000001     0.269474           0.297449       0.262266   
                    0.000010     0.269474           0.297449       0.262266   
                    0.000100     0.269474           0.297449       0.262266   
            0.5     0.000001     0.327647           0.337031       0.353543   
                    0.000010     0.327647           0.337031       0.353543   
                    0.000100     0.327647           0.337031       0.353543   
            0.8     0.000001     0.437257           0.410613       0.399491   
                    0.000010     0.437257           0.410613       0.399491   
                    0.000100     0.437257           0.410613       0.399491   
            0.9     0.000001     0.606010           0.596420       0.604897   
                    0.000010     0.606010           0.596420       0.604897   
                    0.000100     0.606010           0.596420       0.604897   

                                       leiden_ari_0                     \
layers                            [50]    [100, 30] [200, 100, 30, 30]   
temperature dropout lr                                                   
0.01        0.2     0.000001  0.513275     0.297312           0.223355   
                    0.000010  0.513275     0.297312           0.223355   
                    0.000100  0.513275     0.297312           0.223355   
            0.5     0.000001  0.508214     0.306686           0.263858   
                    0.000010  0.508214     0.306686           0.263858   
                    0.000100  0.508214     0.306686           0.263858   
            0.8     0.000001  0.742309     0.256926           0.202755   
                    0.000010  0.742309     0.256926           0.202755   
                    0.000100  0.742309     0.256926           0.202755   
            0.9     0.000001  0.892869     0.243826        

In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 '10X_PBMC_select_2100',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'worm_neuron_cell_select_2100',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'mouse_ES_cell_select_2100',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 'mouse_bladder_cell_select_2100',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_1model.pkl")

In [6]:
df.head()

,dataset,dropout,kmeans_ari_0,kmeans_nmi_0,layers,leiden_ari_0,leiden_nmi_0,lr,run,temperature
0,Quake_Smart-seq2_Trachea,0.8,0.540060,0.650050,[50],0.169197,0.517311,0.00001,0.0,0.07
1,Quake_Smart-seq2_Trachea,0.8,0.529791,0.639803,[50],0.193608,0.535134,0.00001,1.0,0.07
2,Quake_Smart-seq2_Trachea,0.8,0.488717,0.561485,"[100, 30]",0.173371,0.522036,0.00001,0.0,0.07
3,Quake_Smart-seq2_Trachea,0.8,0.466333,0.543368,"[100, 30]",0.190239,0.530519,0.00001,1.0,0.07
4,Quake_Smart-seq2_Trachea,0.8,0.547574,0.672945,"[200, 100, 30]",0.189931,0.536097,0.00001,0.0,0.07


In [ ]:
# df = pd.DataFrame()

for dataset in files[2:]:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = 1500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    for temperature in tqdm([0.07, 0.5, 0.01]):
        for dropout in [ 0.8, 0.2, 0.5, 0.9]:
            for lr in [1e-5, 1e-4, 1e-6]:
                for layers in [[50],[100, 30],  [200, 100, 30],  [200, 100, 30, 30]]:
                    for run in range(2):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=20,
                                             lr=lr,
                                             temperature=temperature,
                                             dropout=dropout,
                                             evaluate=False,
                                             n_ensemble=1,
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["lr"] = lr
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
#                         print(f"# {temperature}, {dropout}, {lr}, {layers}", 
#                               dresults.get('COMBINED_kmeans_ari', ""),
#                               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
#                               dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data_1model.pkl")

>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 1500 genes


................................................................................................................................................................................................................................................................................................
>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 1500 genes


......

In [ ]:
str(layers)

In [8]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"]).mean().unstack("layers")

kmeans_ari_0                                    \
layers                          [100, 30] [200, 100, 30, 30] [200, 100, 30]   
temperature dropout lr                                                        
0.01        0.2     0.000001     0.544060           0.344583       0.439934   
                    0.000010     0.544060           0.344583       0.439934   
                    0.000100     0.544060           0.344583       0.439934   
            0.5     0.000001     0.635475           0.569200       0.478354   
                    0.000010     0.635475           0.569200       0.478354   
                    0.000100     0.635475           0.569200       0.478354   
            0.8     0.000001     0.725628           0.741602       0.750407   
                    0.000010     0.725628           0.741602       0.750407   
                    0.000100     0.725628           0.741602       0.750407   
            0.9     0.000001     0.869916           0.651451       0.782380   
                    0.000010     0.869916           0.651451       0.782380   
                    0.000100     0.869916           0.651451       0.782380   
0.07        0.2     0.000001     0.111859           0.080336       0.075659   
                    0.000010     0.111859           0.080336       0.075659   
                    0.000100     0.111859           0.080336       0.075659   
            0.5     0.000001     0.396924           0.338916       0.398945   
                    0.000010     0.396924           0.338916       0.398945   
                    0.000100     0.396924           0.338916       0.398945   
            0.8     0.000001     0.725727           0.751749       0.743349   
                    0.000010     0.725727           0.751749       0.743349   
                    0.000100     0.725727           0.751749       0.743349   
            0.9     0.000001     0.741593           0.745040       0.747223   
                    0.000010     0.741593           0.745040       0.747223   
                    0.000100     0.741593           0.745040       0.747223   
0.50        0.2     0.000001     0.312590           0.379320       0.326096   
                    0.000010     0.312590           0.379320       0.326096   
                    0.000100     0.312590           0.379320       0.326096   
            0.5     0.000001     0.365930           0.410879       0.422124   
                    0.000010     0.365930           0.410879       0.422124   
                    0.000100     0.365930           0.410879       0.422124   
            0.8     0.000001     0.513746           0.474408       0.456409   
                    0.000010     0.513746           0.474408       0.456409   
                    0.000100     0.513746           0.474408       0.456409   
            0.9     0.000001     0.757611           0.747289       0.761277   
                    0.000010     0.757611           0.747289       0.761277   
                    0.000100     0.757611           0.747289       0.761277   

                                       kmeans_nmi_0                     \
layers                            [50]    [100, 30] [200, 100, 30, 30]   
temperature dropout lr                                                   
0.01        0.2     0.000001  0.620582     0.624383           0.447505   
                    0.000010  0.620582     0.624383           0.447505   
                    0.000100  0.620582     0.624383           0.447505   
            0.5     0.000001  0.599731     0.697316           0.636363   
                    0.000010  0.599731     0.697316           0.636363   
                    0.000100  0.599731     0.697316           0.636363   
            0.8     0.000001  0.842513     0.729092           0.772006   
                    0.000010  0.842513     0.729092           0.772006   
                    0.000100  0.842513     0.729092           0.772006   
            0.9     0.000001  0.899217     0.819211        

In [ ]:
run = 0 
dataset = 'Quake_Smart-seq2_Diaphragm'
dataset = 'Quake_Smart-seq2_Trachea'

In [ ]:
torch.manual_seed(run)
torch.cuda.manual_seed_all(run)
np.random.seed(run)
random.seed(run)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f">>>>> Data {dataset}")

data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
X = np.array(data_mat['X'])
Y = np.array(data_mat['Y'])
print(np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1]))
cluster_number = np.unique(Y).shape[0]
nb_genes = 1500
data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
X = np.array(data_mat['X'])
Y = np.array(data_mat['Y'])

X = train.preprocess(X, nb_genes=nb_genes)

In [ ]:
run = 0
torch.manual_seed(run)
torch.cuda.manual_seed_all(run)
np.random.seed(run)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
res = train.run(X,
        cluster_number,
        dataset,
        Y=Y,
        nb_epochs=20,
        lr=1e-5,
        temperature=0.07,
        dropout=0.8,
        evaluate=True,
        n_ensemble=3,
        layers = [256, 64, 32],
        save_to=f"{path}output/real_data/{run}/")

In [ ]:
res.keys()

In [ ]:
dresults['COMBINED_kmeans_ari'], dresults['COMBINED_leiden_ari'], dresults['kmeans_ari_0'], dresults['leiden_ari_0']

In [ ]:
df = df.append(res, ignore_index=True)

In [ ]:
df

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')